<a href="https://colab.research.google.com/github/mohamedaichaouiii/Movie-recommendation-system-with-Python-and-Pandas/blob/main/Movie_recommendation_system_with_Python_and_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hello everyone, My name is Mohamed Aichaoui and this is my project intitled Movie recommendation system with Python and Pandas. 
Feel free to contact me if there is any advisees.
Let's start!


In [1]:
import pandas as pd
movies = pd.read_csv("movies.csv")

In [2]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [3]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

In [4]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [5]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))#Read one or two words in a row to make the search more precise.
tfidf = vectorizer.fit_transform(movies["clean_title"])

Compute the similarity between the term that we enter and all the movies in our list

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])# turn the title in a set of number
    similarity = cosine_similarity(query_vec, tfidf).flatten()#calculate the similarity
    indices = np.argpartition(similarity, -5)[-5:]#the indice of the most 5 similar title
    results = movies.iloc[indices].iloc[::-1]#the title 
    
    return results

Building an interactive search box 

In [14]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

The second part : Recommendation

In [15]:
ratings = pd.read_csv("ratings.csv")

In [16]:
ratings


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
24930890,162132,356,3.0,1202095829
24930891,162132,377,2.5,1212265315
24930892,162132,380,2.5,1202097754
24930893,162132,442,1.5,1212537377


In [46]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [47]:
movie_id = 89745

#def find_similar_movies(movie_id):
movie = movies[movies["movieId"] == movie_id]

In [48]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [49]:
similar_users

array([    21,    187,    208, ..., 162088, 162102, 162108])

In [50]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]#user like the same movie as us and ration 5 star

In [51]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]#movie greater tthan 10% like 

In [52]:
similar_user_recs

89745     1.000000
58559     0.572593
59315     0.530351
79132     0.519042
7153      0.496591
            ...   
106072    0.103609
780       0.103110
88744     0.102777
1258      0.100948
1193      0.100782
Name: movieId, Length: 193, dtype: float64

Finding how mush all users like movies

In [53]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [54]:
all_users

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
29,1,4973,4.5,1147869080
48,1,7361,5.0,1147880055
72,2,110,5.0,1141416589
76,2,260,5.0,1141417172
...,...,...,...,...
24930797,162130,593,5.0,1042948233
24930832,162131,1200,5.0,1237597137
24930849,162131,7438,5.0,1237593562
24930870,162132,110,5.0,1202086520


In [55]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [56]:
all_user_recs

318       0.346240
296       0.288106
2571      0.246939
356       0.238096
593       0.228607
            ...   
86332     0.010147
91630     0.009300
122900    0.008548
122926    0.008077
106072    0.005275
Name: movieId, Length: 193, dtype: float64

Creating a recommendation score

In [57]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [58]:
rec_percentages

,similar,all
89745,1.000000,0.040407
58559,0.572593,0.148115
59315,0.530351,0.054815
79132,0.519042,0.132975
7153,0.496591,0.174598
...,...,...
106072,0.103609,0.005275
780,0.103110,0.054728
88744,0.102777,0.010355
1258,0.100948,0.083852


In [59]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [60]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [61]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
17067,1.000000,0.040407,24.748046,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX,Avengers The 2012
20513,0.103609,0.005275,19.640806,106072,Thor: The Dark World (2013),Action|Adventure|Fantasy|IMAX,Thor The Dark World 2013
25058,0.240812,0.012345,19.507442,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi,Avengers Age of Ultron 2015
19678,0.216365,0.012109,17.867485,102125,Iron Man 3 (2013),Action|Sci-Fi|Thriller|IMAX,Iron Man 3 2013
16725,0.215034,0.012042,17.856709,88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War,Captain America The First Avenger 2011
16312,0.175786,0.010147,17.323632,86332,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,Thor 2011
21348,0.287211,0.016713,17.185314,110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX,Captain America The Winter Soldier 2014
25071,0.213205,0.012808,16.645852,122920,Captain America: Civil War (2016),Action|Sci-Fi|Thriller,Captain America Civil War 2016
25061,0.135872,0.008548,15.895561,122900,Ant-Man (2015),Action|Adventure|Sci-Fi,AntMan 2015
14628,0.242641,0.015496,15.658022,77561,Iron Man 2 (2010),Action|Adventure|Sci-Fi|Thriller|IMAX,Iron Man 2 2010


Building a recommendation function

In [62]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [63]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()